# Introduction

k- nearest neighbors classifier is a model that consist only of storing the training set and make prediction for a new data point finding the point in the training set that is closest to the new point.Then it assigns the label of this training point to the new data point. It relies on a chosen distance metric (e.g., Euclidean, Manhattan) to find nearest neighbors.

The k in k-nearest neighbors signifies that instead of using only the closest neighbor to the new data point, the model can be fixed with any number "k" of neighbors in the training.


The goal of this post is to build a k-Nearest Neighbors model from scratch.  The development process will involve the following steps:

1 - Calculate distance.

2 - Get nearest neighbors.

3 - Make predictions


For categorical data, where the outcome is represented by distinct categories, this algorithm determines the most frequent class among the k nearest neighbors.

For continuous data, where the outcome is a numerical value, two alternative prediction methods can be employed:

Inverse Distance Weighting: In this method, closer neighbors are assigned higher weights, giving them more influence on the prediction. The weights are inversely proportional to the distance from the new data point.

Exponential Distance Weighting: This method assigns exponentially decaying weights to neighbors as their distance from the new data point increases.


# The data set

For classification, we will utilize the Iris dataset from scikit-learn. This dataset comprises measurements of petal and sepal length for three different types of irises: Setosa, Versicolour, and Virginica. The data is stored in a 150x4 NumPy array.

For regression, we will employ the Diabetes dataset from scikit-learn. The target variable in this dataset consists of integer values ranging from 25 to 346.




# Code

Import libraries

In [1]:
import numpy as np
from sklearn import datasets


Create a function that randomly divides an array or matrix into train and test subsets. This step could be accomplished using the train_test_split function from the sklearn.model_selection module in a single line of code.

However, implementing it from scratch provides a deeper understanding of the data splitting process and allows for customization based on specific requirements.



In [2]:
def train_test_random_split(p_train,X,Y):
    assert 0 < p_train < 1, 'ratio must be between zero and one'
    data_len = len(X) #data lenght
    train_len = round(p_train*data_len) #get integer number
    perm = np.random.permutation(data_len) #generates random numbers
    train_pos = perm[:train_len] #take random numbers from 0 to desire train lenght
    test_pos = perm[train_len:] #same for test
    return X[train_pos], Y[train_pos], X[test_pos], Y[test_pos] #ste values


Object oriented k-nearest code

In [3]:

class KNN:

    def __init__(self,x_train,y_train,k,classification_mode=True,\
                 weight_function='ones'):
        self.X_train = x_train
        self.Y_train = y_train
        self.k = k
        self.classification_mode = classification_mode
        self.weight_function = weight_function


    def _distance(training_data, point):
      #calculates the Euclidean distance between each data point
        return np.linalg.norm(training_data - point,axis=1)

    def _k_neighbours(self,point):
       #all the stuff declare at self and the new external point
        distances = KNN._distance(self.X_train,point) #get distance all data set
        order_of_distances = distances.argsort() #sort distances in ascending order
        sorted_distances = distances[order_of_distances] #sort the actual distances along the indices
        sorted_labels = self.Y_train[order_of_distances] #get the labels of the sorted points
        return sorted_labels[:self.k], sorted_distances[:self.k] #return the k first labels and distances

    def _majority_vote(self,y_labels,distances):

        if self.classification_mode: #schecks if the KNN is used for classification
            votes, freq = np.unique(y_labels, return_counts=True) #finds the unique values (classes)
            most_common_val = votes[np.argmax(freq)] #finds the most frequent
            prediction = most_common_val #extracts the corresponding class label

        else: #regression (predicting continuous values) 3 mehtods
            if self.weight_function == 'inverse': #gives closer neighbors more weight
                eps = 0.1
                weights = 1/(distances+eps)
            elif self.weight_function == 'exponential': #assigns exponentially decaying weights based on distance,
                weights = np.exp(-distances)
            else:
                weights = np.ones_like(distances) #uses equal weights for all neighbors.
            weighted_avg = np.average(y_labels, weights=weights) #calculates a weighted average of the neighbor labels using the chosen weights.
            prediction = weighted_avg #sets the predicted value
        return prediction

    def predict(self,point):
      #Predicts the class or value for a new data point based on the majority vote of its k nearest neighbors.
        sorted_labels, sorted_distances = self._k_neighbours(point)
        return self._majority_vote(sorted_labels, sorted_distances)

    def _predict_on_batch(self, X_test):
       #make predictions for  X-test
        return np.array([self.predict(test_row) for test_row in X_test])

    def score(self,y_test,X_test,type_data): #compara resultado de Xtest con el valor real, y_teest
        y_calculated = self._predict_on_batch(X_test) #lista de resultados calculados
        assert len(y_test) == len(y_calculated),'labels and predictions must be the same length' #assert ensures the number of true labels and predicted labels are the same.

        if self.classification_mode:
          #This block handles the scenario where the model is predicting discrete classes (classification).

            score = (y_test == y_calculated).sum()/len(y_test) #calculates the accuracy
            print('Success percentage on '+type_data+' {:.2%}\n'.format(score))

        else:
           #here it deals with regression problems where the model predicts continuous values

            score = np.sqrt( ((y_test-y_calculated)**2).mean() ) #measure of the differences between values predicted by a model and the values observed in reality.
            print('average error on '+type_data+' {:.4}\n'.format(score))
        # print(f'The score is {score}')
        return score




Test code for clasification

In [12]:
IRIS = True
DIABETES = False

In [14]:
if __name__ == '__main__':
    if IRIS:
        dataset = datasets.load_iris()
        classify = True
    if DIABETES:
        dataset = datasets.load_diabetes()
        classify = False

    X_train, Y_train, X_test, Y_test = train_test_random_split(0.7,dataset.data, dataset.target)

    knn = KNN(X_train, Y_train, 3,classification_mode=classify,weight_function='exponential')

    score_train = knn.score(Y_train,X_train,'train')
    score_test = knn.score(Y_test,X_test,'test')





Success percentage on train 96.19%

Success percentage on test 95.56%



Using sklearn library

In [28]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score

dataset = datasets.load_iris()
X_train, Y_train, X_test, Y_test = train_test_random_split(0.7,dataset.data, dataset.target)


knn = KNeighborsClassifier(n_neighbors=3)
knn.fit(X_train, Y_train)
y_pred = knn.predict(X_test)
accuracy = accuracy_score(Y_test, y_pred)
print("Accuracy:", accuracy)

Accuracy: 0.9555555555555556


**We arrive to the same Accuracy than the one obtained in clasification mode.**

Changing to regresion


In [9]:
IRIS = False
DIABETES = True

Setting weight_function='exponential'

In [10]:
if __name__ == '__main__':
    if IRIS:
        dataset = datasets.load_iris()
        classify = True
    if DIABETES:
        dataset = datasets.load_diabetes()
        classify = False

    X_train, Y_train, X_test, Y_test = train_test_random_split(0.7,dataset.data, dataset.target)

    knn = KNN(X_train, Y_train, 3,classification_mode=classify,weight_function='exponential')

    score_train = knn.score(Y_train,X_train,'train')
    score_test = knn.score(Y_test,X_test,'test')

average error on train 44.69

average error on test 62.24



weight_function='inverse'

In [11]:
if __name__ == '__main__':
    if IRIS:
        dataset = datasets.load_iris()
        classify = True
    if DIABETES:
        dataset = datasets.load_diabetes()
        classify = False

    X_train, Y_train, X_test, Y_test = train_test_random_split(0.7,dataset.data, dataset.target)

    knn = KNN(X_train, Y_train, 3,classification_mode=classify,weight_function='inverse')

    score_train = knn.score(Y_train,X_train,'train')
    score_test = knn.score(Y_test,X_test,'test')

average error on train 37.19

average error on test 54.96



Based on the high error rates in both inverse and exponential weights, it can be concluded that the current model is not well-suited for making these types of predictions.

In both cases, the error in prediction test is high. We can conclude that ithis is not a good model to make this predictions.